In [9]:
pip install pyperclip

  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11112 sha256=b8b4da8692f830a6ca08092fc10b5f888c26822b68d2075479a500d057926e79
  Stored in directory: c:\users\scarl\appdata\local\pip\cache\wheels\7f\1a\65\84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip
Note: you may need to restart the kernel to use updated packages.


In [10]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait as wait
import pandas as pd
import time
import pyperclip

In [45]:
def crawler():

#     naver_id = "hongseoljin"
#     naver_pwd = "garam8893**"
    search_artist = input("검색할 가수명: ")
    #로그인을 안하면 19금 걸린 노래를 못들어가서 로그인 필요!

    driver = webdriver.Chrome(
        executable_path = '/Users/scarl/anaconda3/pkgs/python-chrome-driver-binary-92.0.4515.107/chromedriver') 
    #이부분은 각자의 크롬드라이버 절대경로 설정해야해!
    driver.maximize_window()
    vibe = "https://vibe.naver.com/"
    driver.get(vibe)
    popup = driver.find_element_by_xpath('//*[@id="app"]/div[2]/div/div/a[2]')
    popup.click()
    #지금 무슨 이벤트하는거같아서 팝업이 뜨는거같은데 만약 나중에 이벤트 끝나면 위 두라인은 주석처리하면 돼!

    loginbutton = driver.find_element_by_xpath('/html/body/div/div/header/div[2]/div[1]/a/span')
    loginbutton.click()
    
    tag_id = driver.find_element_by_name('id')
    tag_pw = driver.find_element_by_name('pw')
    tag_id.clear()
    time.sleep(1)
    
    tag_id.click()
    pyperclip.copy('네이버 id') # 요기랑 (1) - 세린갓
    tag_id.send_keys(Keys.CONTROL, 'v')
    time.sleep(1) 
    
    # pw 입력 
    tag_pw.click()
    pyperclip.copy('네이버 password') # 요기만 (2) 바꾸면 댑니당 ! ! ! - 세린갓
    tag_pw.send_keys(Keys.CONTROL, 'v')
    time.sleep(1)
    
    # 로그인 버튼을 클릭합니다 
    login_btn = driver.find_element_by_id('log.login') 
    login_btn.click()

    # 자주쓰는 기기 등록 나오길래 나는 필요했었는데 필요없으면 주석처리하면 될 것 같아 !!!  - 석민
    enter=driver.find_element_by_id('new.save')
    enter.click()
    
    #여기가 로그인하는부분

    time.sleep(1)
    searchbutton = driver.find_element_by_xpath('//*[@id="header"]/a[1]')
    searchbar = driver.find_element_by_xpath('//*[@id="search_keyword"]')
    searchbutton.click()
    searchbar.click()

    searchbar.send_keys(search_artist)
    searchbar.send_keys(Keys.ENTER)
    searchbar.send_keys(Keys.ESCAPE)
    #여기가 검색 입력하는 부분

    wait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div[1]/div/a/div[2]/div[1]/div'))).click()
    time.sleep(1)

    # section_title이 다른 부분에도 적용돼서 노래 목록으로 넘어갈 수 있게 하는 코드 !!! - 석민
    
    songs = driver.find_elements_by_class_name('section_title') 
    for i in range(5): 
        if '노래' in  songs[i].text:
            songs[i].click()
            break
       
    #가수의 노래 목록 들어가는 부분

    time.sleep(1)
    while True:
        try:
            showmore = driver.find_element_by_xpath('/html/body/div/div/div[3]/div/div/div[3]/div[2]/a/span')
            showmore.click()
        except :
            break
    #곡이 많은 가수들은 곡 더보기를 눌러줘야 전체 목록이 뜨더라고! 아닌 가수는 그냥 패스

    nums = list(range(1, 500))
    links = []

    time.sleep(1)

    for num in nums:
        try:
            links.append(driver.find_element_by_xpath(f'//*[@id="content"]/div/div[3]/div[1]/div/\
                table/tbody/tr[{num}]/td[3]/div[1]/span/a').get_attribute('href'))

        except NoSuchElementException:
            break
    #여기가 모든 노래의 링크 긁어오는 부분

    linknumbers = [link[29:] for link in links]
    #링크의 중요한 키만 남기고 지우는 부분

    lyrics = []
    titles = []

    for linknumber in linknumbers:
        try:
            driver.get("https://vibe.naver.com/track/" + str(linknumber))
            time.sleep(0.5)
            title = driver.find_element_by_class_name("title")
            lyric = driver.find_element_by_class_name("lyrics")
            titles.append(title.text)
            lyrics.append(lyric.text)
            time.sleep(0.5)

        except : # 그냥 except으로 바꿔도 되길래 바꾸긴했는데 아니다 싶으면 다시 고쳐도 될듯! - 석민
            continue
    #여기서 실제로 모든 링크 하나씩 들어가면서 제목이랑 가사 긁어와줘!

    titles = [_[3:] for _ in titles]
    #제목 "곡명\n" 지우는 전처리

    lyrics_new = []
    for i in lyrics:
        lyrics_new.append(i.replace("\n", " "))
    lyrics_new
    #가사마다 행변환 부분 삭제

    crawled = pd.DataFrame({"제목": titles, "가사": lyrics_new})
    crawled.to_excel(f'{search_artist} 가사 crawling.xlsx', encoding='utf-8')
    #저장!!

if __name__ == '__main__':
    crawler()

검색할 가수명: 김효은
